In [7]:
import pandas as pd

df = pd.read_csv(r"D:\PHUCHUY\THUC_HANH\dataset.csv\dataset.csv")  # Đặt 'r' trước đường dẫn để tránh lỗi escape characters
print(df.columns)  # Xem danh sách cột


Index(['app_id', 'app_name', 'review_text', 'review_score', 'review_votes'], dtype='object')


In [ ]:
# Cài đặt các thư viện cần thiết
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Tải dữ liệu NLP cần thiết
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Danh sách từ sentiment theo rule-based
positive_words = {"good", "great", "excellent", "amazing", "fantastic", "love", "wonderful", "best", "awesome", "happy"}
negative_words = {"bad", "terrible", "awful", "worst", "hate", "poor", "horrible", "disappointed", "sad", "annoying"}

# Hàm tiền xử lý văn bản
def preprocess_text(text):
    text = str(text).lower()  # Chuyển về chữ thường
    text = re.sub(r'[^a-zA-Z ]', '', text)  # Loại bỏ ký tự không phải chữ cái
    words = word_tokenize(text)  # Tách từ
    words = [w for w in words if w not in stopwords.words('english')]  # Bỏ stopwords
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(w) for w in words]  # Lemmatization
    return " ".join(words)

# Hàm phân loại sentiment theo rule-based
def rule_based_sentiment(text):
    words = set(word_tokenize(text.lower()))
    pos_count = len(words & positive_words)
    neg_count = len(words & negative_words)
    if pos_count > neg_count:
        return "positive"
    elif neg_count > pos_count:
        return "negative"
    else:
        return "neutral"

# Đọc dữ liệu từ file CSV
df = pd.read_csv(r"D:\PHUCHUY\THUC_HANH\dataset.csv\dataset.csv", encoding="utf-8")

# Kiểm tra cột có sẵn
print("Các cột trong file CSV:", df.columns)

# Kiểm tra xem cột 'review_text' có tồn tại không
if 'review_text' not in df.columns or 'review_score' not in df.columns:
    raise KeyError("File CSV phải có cột 'review_text' và 'review_score'. Kiểm tra lại dữ liệu!")

# Áp dụng tiền xử lý và rule-based sentiment
df['cleaned_review'] = df['review_text'].apply(preprocess_text)
df['rule_sentiment'] = df['cleaned_review'].apply(rule_based_sentiment)

# Chia tập dữ liệu train-test
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_review'], df['review_score'], test_size=0.2, random_state=42)

# Mô hình Machine Learning (RandomForest)
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),   # Chuyển văn bản thành vector đếm từ
    ('tfidf', TfidfTransformer()),       # Chuyển thành trọng số TF-IDF
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))  # Mô hình RF
])

# Huấn luyện mô hình
pipeline.fit(X_train, y_train)
y_pred_ml = pipeline.predict(X_test)

# So sánh với rule-based
y_pred_rule = df.loc[X_test.index, 'rule_sentiment']

# Kết hợp Rule-based + ML (Voting)
def hybrid_sentiment(rule_label, ml_label):
    if rule_label != "neutral":  # Nếu rule-based đưa ra kết quả rõ ràng thì giữ nguyên
        return rule_label
    return ml_label  # Nếu rule-based là 'neutral', dùng ML

df.loc[X_test.index, 'hybrid_sentiment'] = [hybrid_sentiment(r, m) for r, m in zip(y_pred_rule, y_pred_ml)]

# Đánh giá kết quả
print("\n=== Đánh giá mô hình ===")
print("Machine Learning Accuracy:", accuracy_score(y_test, y_pred_ml))
print("Rule-based Accuracy:", accuracy_score(y_test, y_pred_rule))
print("Hybrid Accuracy:", accuracy_score(y_test, df.loc[X_test.index, 'hybrid_sentiment']))
print("\nClassification Report (Hybrid Model):")
print(classification_report(y_test, df.loc[X_test.index, 'hybrid_sentiment']))

# Xuất kết quả ra file CSV
df.to_csv("sentiment_results.csv", index=False)
print("Kết quả đã được lưu vào sentiment_results.csv")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\phuy7\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\phuy7\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\phuy7\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
